In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
from scdesigner.experimental.estimators import negative_binomial_regression
from scdesigner.experimental.transform import nullify, amplify, data_frame_mask

params = negative_binomial_regression(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
outcomes = example_sce.var_names[:4]
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = nullify(params, "coefficient", mask)
null_params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.733727,1.689458,1.651232,1.555795,2.025110,1.444613,1.584104,1.572423,1.389908,1.395772,...,0.622493,-0.016562,1.325979,0.732366,1.021546,-0.071549,0.967869,1.721433,-1.769911,-0.324306
"bs(pseudotime, degree=2)[1]",0.000000,0.000000,0.000000,0.000000,1.550395,1.539665,1.716174,0.460826,0.275353,-1.156778,...,0.849135,1.638570,1.285707,2.082242,-1.730549,2.030675,1.460706,-0.683348,-2.834935,2.759499
"bs(pseudotime, degree=2)[2]",0.000000,0.000000,0.000000,0.000000,-3.783759,1.599531,1.261496,1.685368,1.671481,1.665799,...,0.968832,0.368252,-1.606683,-3.385910,1.537826,0.048395,-1.933420,-3.407623,2.610776,-1.205206


In [3]:
from scdesigner.experimental.samplers import negative_binomial_regression_sample

samples = negative_binomial_regression_sample(params, example_sce.obs, "~ bs(pseudotime, degree=2)")
samples.X[:10, :10]

array([[77, 14, 39, 19,  3,  4, 10, 23,  0,  2],
       [ 5,  7, 10,  1,  3,  5,  9,  5, 12,  3],
       [29, 32,  1,  5,  0,  7,  7, 12,  2,  2],
       [83, 23, 30, 12,  0, 16, 10, 13,  8,  8],
       [ 9,  1,  2,  4,  1, 12,  3,  5,  8,  3],
       [12,  5, 15, 10, 18,  3, 15,  6,  2, 11],
       [ 0,  5,  6,  9,  4, 11,  4,  2, 16,  2],
       [41, 36,  4, 18,  2, 18, 25,  3,  4,  0],
       [40,  6,  0, 31,  0, 17, 10, 19, 11, 20],
       [ 1, 23, 17, 12,  3,  6, 76,  4,  4, 13]])

In [4]:
from scdesigner.experimental.estimators import negative_binomial_copula

params = negative_binomial_copula(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
mask = data_frame_mask(params["covariance"], outcomes)
null_params = nullify(params, "covariance", mask)
null_params["covariance"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Iapp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chgb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Rbp4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Spp1,0.388931,0.794727,-0.291704,0.305696,2.391053,-0.238807,-0.900809,0.541680,0.405472,0.312711,...,-0.093324,-0.073809,-0.014566,-0.202387,0.417031,-0.034419,-0.106412,0.029012,-0.117047,-0.020695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,-0.002059,0.019737,0.241674,-0.000016,-0.034419,0.137606,0.004483,0.023260,0.024142,0.041231,...,0.027136,0.088966,-0.034760,0.021598,0.004471,0.842930,-0.010285,-0.025092,-0.009528,0.049784
Hes6,0.309318,0.647835,-0.104181,0.162515,-0.106412,-0.082867,-0.038989,0.368711,0.381719,0.343955,...,0.169445,-0.028254,0.405345,0.415623,0.175142,-0.010285,1.163770,0.347013,-0.030266,0.069241
Serpinh1,0.447753,0.651780,-0.061203,0.287469,0.029012,0.003541,-0.161416,0.389771,0.372379,0.373852,...,0.077674,-0.009025,0.297558,0.349412,0.158698,-0.025092,0.347013,1.069366,-0.032348,0.079767
Npy,-0.403169,-0.250708,-0.165001,-0.304390,-0.117047,-0.116625,0.229695,0.070518,-0.027507,0.041941,...,0.090407,-0.005050,0.015866,-0.050693,0.049929,-0.009528,-0.030266,-0.032348,0.443379,-0.027355


In [5]:
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = amplify(params, "coefficient", mask, factor=2)
null_params["coefficient"] / params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[1]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[2]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
from scdesigner.experimental.samplers import negative_binomial_copula_sample

negative_binomial_copula_sample(params, example_sce.obs, "~ bs(pseudotime, degree=2)")

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'